In [2]:
from tqdm import tqdm 
import os
import json
from colorama import Style,Fore
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
def openJson(path):
    with open(path, "r", encoding="utf-8") as file:
        data = json.load(file)
    return data

def saveJson(path,data):
    with open(path, "w", encoding="utf-8") as f:
       json.dump(data, f, ensure_ascii=False, indent=2)
       print(Style.BRIGHT+Fore.GREEN+'\n json saved'+Style.RESET_ALL)

# 1 Vocabulaire grains

- Un vocabulaire identifié manuellement va être utilisé comme base pour une extension.

In [9]:
keywordsv0 = openJson('./jsons/keywords.json')
len(keywordsv0)

5

# 2 Extension avec un LLM

- On va faire une extension en deux parties : la première en se basant sur les métadonnées d'une vidéo, et la deuxième sur les commentaires.

## 2.1 Extension basée sur les vidéos

- On va traiter ici que les 4 premières thématiques : maladies\_ravageurs, eau, sol, adventices (la thématique récoltes est exclusive aux commentaires, donc on la laissera à la deuxième extension).

In [10]:
from langchain_core.prompts import ChatPromptTemplate

In [11]:
from langchain_openai import ChatOpenAI

gpt_4_1_mini = ChatOpenAI(
    model="openai/gpt-4.1-mini",
    temperature=0,
    max_retries=2,
    api_key=os.getenv("GITHUB_TOKEN_ITSAIDI"),
    base_url="https://models.github.ai/inference",
)

In [12]:
sysPrompt = """
- Tu es un expert en extraction de mots clés.

# Les thématiques :
---

##------------------------- Maladies_Ravageurs

### Définition :
 
Cette classe inclut les actions mises en place pour gérer les maladies et ravageurs des plantes cultivées (arbres compris), comme les limaces, les pucerons, les maladies fongiques, etc. 
Peut inclure, par exemple, des actions telles que le traitement avec bouillie bordelaise, les traitements par des purins, le lâcher de poules ou de canards indiens, le choix des porte-greffes ou des variétés résistantes, 
l’organisation spatiale des cultures pour éviter la concentration d’une même espèce, la plantation d’espèces végétales réputées protectrices.

### Exemples de mots clés

Ravageur, limace, puceron, maladie fongique, bouillie bordelaise, purin, lâcher de poules, lâcher de canards indiens, porte-greffe, espèce végétale protectrice.

##------------------------- Eau 

### Définition :

Cette classe inclut les actions mises en place pour gérer l’eau dans la production agricole – qu’il s’agisse d’accès à l’eau (puits, source, réseau d’eau potable, ...), 
de manque d’eau (sécheresse...), d’excès d’eau (sols hydromorphes...), les économies d’eau pour éviter le gaspillage et réduire les coûts. 
Ces actions peuvent être, par exemple, la mise en place de paillage pour éviter l’évaporation en été, l’irrigation par goutte-à-goutte, etc.


### Exemples de mots clés

Puits, réseau d’eau potable, sécheresse, sol hydromorphe, paillage, évaporation, irrigation, goutte-à-goutte.

##------------------------- Sol

### Définition :
 
Cette classe inclut des actions mises en place pour rendre le sol le plus en adéquation possible avec les besoins des plantes cultivées, 
en termes de structure, composition et fertilité. Quelques exemples de telles actions sont l’apport d’engrais organique ou pas (fumiers divers, engrais de synthèse ; apport de compost) ; l’apport de biomasse en paillage (paille, foin, broyat, BRF, feuilles mortes, branchages coupés pour cet objectif) ; 
favoriser le vivant dans le sol ; pratiquer la perturbation des végétaux « sacrificiels » pour stimuler les végétaux autour d’eux.


### Exemples de mots clés

Fertilité, engrais, engrais organique, fumiers, biomasse, engrais de synthèse, compost, paille, foin, broyat, brf, feuilles mortes, branchages coupés.

##------------------------- Adventices 

### Définition :
 
Cette classe inclut des actions mises en place pour gérer les végétaux indésirables, aussi nommés « mauvaises herbes » ou adventices. 
Des exemples incluent l’utilisation de bâches en tissu renforcé, la mise en place de couverts végétaux, la densification des cultures, le désherbage manuel, mécanique ou chimique.


### Exemples de mots clés

Végétal indésirable, mauvaise herbe, couvert végétaux, densification des cultures, désherbage.

-->

\n

# Ta mission :
---
- À partir d’un contexte composé de métadonnées issues d’une ou plusieurs vidéos YouTube 
(titre, description, tags), tu dois extraire les mots clés appartenant à chacune des 4 thématiques prédéfinies.

\n

# Ta réponse :
---
- Ta réponse doit être un objet JSON.
- Les clés du JSON sont les 4 thématiques en minuscules.
- La valeur associée à chaque clé est une liste de mots clés spécifiques à cette thématique.

\n

# Consignes :
---
- N’ajoute aucune explication, seulement le JSON en sortie.
- Limite l’extraction des mots clés strictement au contexte fourni.
- Ne crée pas de mots clés génériques ou hors-sujet.
"""

userprompt = """
Le Contexte
---

{contexte}
"""

prompt = ChatPromptTemplate([
    ("system", sysPrompt),
    ("user", userprompt)
])

chain = prompt | gpt_4_1_mini

### 2.1.1 Prepare the Input

In [44]:
def getContext(videosMetadata):
    contexte = ""    
    for video in videosMetadata:
        titre = video.get('titre','')
        description = video.get('description','')
        tags = ''
        if video['tags']:
            tags = ", ".join(video['tags'])       
        contexte += f"""
# Video
---
## Titre

{titre}

## Description

{description}

## Tags

{tags}
"""
    return contexte
        

- test getContext

In [45]:
videos_test = [
    {
        "titre": "Récolte des pommes en permaculture",
        "description": "Dans cette vidéo, nous découvrons comment récolter des pommes sans perturber l'écosystème du verger.",
        "tags": None
    },
    {
        "titre": "Gérer l'eau en agriculture",
        "description": "Techniques d'irrigation économes et astuces pour optimiser l’eau en période de sécheresse.",
        "tags": ["eau", "irrigation", "sécheresse", "agriculture durable"]
    }
]
print(getContext(videos_test))


# Video
---
## Titre

Récolte des pommes en permaculture

## Description

Dans cette vidéo, nous découvrons comment récolter des pommes sans perturber l'écosystème du verger.

## Tags



# Video
---
## Titre

Gérer l'eau en agriculture

## Description

Techniques d'irrigation économes et astuces pour optimiser l’eau en période de sécheresse.

## Tags

eau, irrigation, sécheresse, agriculture durable



In [46]:
import psycopg

conn = psycopg.connect(
    dbname="youtubestay",
    user="postgres",
    password=os.getenv("POSTGRE_PASSWORD"),
    host="localhost",
    port="5432"
)

In [47]:
cur = conn.cursor()

query = """
select * from videos 
order by date_publication desc 
limit 5
"""

cur.execute(query)
rows = cur.fetchall()

column_names = [desc[0] for desc in cur.description] 

videos_metadata = [dict(zip(column_names, row)) for row in rows]

cur.close()
conn.close()

In [48]:
videos_metadata

[{'id_video': '8q3Bq2l015I',
  'titre': 'La REINE du jardin est dans la place ! Etat des lieux après plantation. #jardin #nature #garden',
  'description': 'Bonjour mes HONORABLES et mes PETITES MAINS VERTES. la mise en place est quasiment terminée pour cet été. ça fait chaud au coeur.\n#tomate #hautsdefrance #nord',
  'date_publication': datetime.date(2025, 5, 20),
  'duree': 610,
  'miniature': 'https://i.ytimg.com/vi/8q3Bq2l015I/maxresdefault.jpg',
  'langue': 'fr',
  'transcription': None,
  'tags': None,
  'requetes': None,
  'id_chaine': 'UCIXF82JbHvwuNmqC1QxJ6eg',
  'id_categorie': None},
 {'id_video': 'a0TffjqYebc',
  'titre': "Isolation d'une Maison Conteneur en Été et Hiver",
  'description': "Isolation d'une Maison Conteneur en Été et Hiver \n.\n#Isolation #Construction #Autonomie",
  'date_publication': datetime.date(2025, 5, 20),
  'duree': 62,
  'miniature': 'https://i.ytimg.com/vi/a0TffjqYebc/maxresdefault.jpg',
  'langue': 'fr',
  'transcription': None,
  'tags': None,


In [50]:
print(getContext(videos_metadata))


# Video
---
## Titre

La REINE du jardin est dans la place ! Etat des lieux après plantation. #jardin #nature #garden

## Description

Bonjour mes HONORABLES et mes PETITES MAINS VERTES. la mise en place est quasiment terminée pour cet été. ça fait chaud au coeur.
#tomate #hautsdefrance #nord

## Tags



# Video
---
## Titre

Isolation d'une Maison Conteneur en Été et Hiver

## Description

Isolation d'une Maison Conteneur en Été et Hiver 
.
#Isolation #Construction #Autonomie

## Tags



# Video
---
## Titre

Incroyable tunnel potager : que récolte-t-on en ce moment ?

## Description

Pénétrez dans mon tunnel potager où une belle diversité de légumes s'épanouit ! De la terre à la récolte, je vous montre mes astuces et les premiers fruits de mon travail.
Rejoignez cette chaîne pour bénéficier d'avantages exclusifs :
https://www.youtube.com/channel/UCFiG3Q_TJpEci70-ovCw92w/join

Apprendre, comprendre et réussir votre jardin au nord de la France pour les débutants.


Mes meilleurs conse